In [13]:
import openpyxl
import xlrd
import regex as re
from openpyxl import workbook

In [14]:
wb_matches = openpyxl.load_workbook('all_matches.xlsx')
wb_batsman= openpyxl.load_workbook('batsman_data.xlsx')
wb_bowler = openpyxl.load_workbook('bowler_data.xlsx')

In [15]:
matches=wb_matches.active
batsman=wb_batsman.active
bowler=wb_bowler.active

In [16]:
#adding data to dictionary
batsmen=dict()
bowlers=dict()
for rowx in range(2, batsman.max_row+1):
    player_name1 = str(batsman.cell(row=rowx, column=1).value)
    player_name = player_name1[0:1] +" "+ player_name1[len(player_name1)-5 : len(player_name1)+1]
    batsmen.update({player_name:rowx})
    
for rowx in range(2,bowler.max_row+1):
    player_name1 = str(bowler.cell(row=rowx,column=1).value) 
    player_name = player_name1[0:1] +" "+ player_name1[len(player_name1)-5 : len(player_name1)+1]
    
    bowlers.update({player_name:rowx})

In [17]:
#cell to add dictionary of batsmen_col and bowler_col
batsmen_col=dict()
bowlers_col=dict()
for colx in range(2,batsman.max_column+1):
    key= str(batsman.cell(row=1,column=colx).value)
    batsmen_col.update({key:colx})
for colx in range(2,bowler.max_column+1):
    key = str(bowler.cell(row=1,column=colx).value)
    bowlers_col.update({key:colx})

In [20]:
#check player's name from all matches in dict and get related data transfer
def check_batsman(player_name):
    #change player_name into dictionary desired name
    player_name1 = str(player_name)
    player_name2 = player_name1[0:1] +" "+ player_name1[len(player_name1)-5 : len(player_name1)+1]
    #try:
    value = batsmen.get(player_name2)
    if value is not None:
        return value
    else:
        data_row=batsman.max_row+1
        batsman.cell(row=data_row, column=1,value= player_name)
        batsmen.update({player_name2: data_row})            
        value = batsmen.get(player_name2)
        return value
        
def check_bowler(player_name):
    player_name1 = str(player_name)
    player_name2 = player_name1[0:1] +" "+ player_name1[len(player_name1)-5: len(player_name1)+1]
    
    #try:
    value = bowlers.get(player_name2)
    if value is not None:
        return value
    else:
        data_row=bowler.max_row+1
        bowler.cell(row=data_row, column=1,value= player_name)
        bowlers.update({player_name2 :data_row})
        value = bowlers_col.get(player_name2)
        return value
        
def update_batsman(b,col,val):
    rowx = check_batsman(b)
    if rowx is not None:
        raw_data = batsman.cell(row=rowx, column=batsmen_col.get(col)).value
        if raw_data is not None:
            data=int(raw_data)
        else:
            data=0
        data+= int(val)
        batsman.cell(row=rowx, column=batsmen_col.get(col),value=data)
        
def update_bowler(b,col,val):
    rowx= check_bowler(b)    
    if rowx is not None:
        raw_data= bowler.cell(row=rowx, column=bowlers_col.get(col)).value
        if raw_data is not None:
            data = int(raw_data)
        else:
            data=0
        data+=int(val)
        bowler.cell(row=rowx, column=bowlers_col.get(col),value=data)

In [21]:
#player change data from am to bd
i,wkt,w=0,0,0
bs,bns,bw="","",""
blist={bs,bns}
for rowx in range(94550,matches.max_row+1):
    year= int(str(matches.cell(row=rowx,column=2).value)[0:4])
    if  2014<= int(year)< 2019:
        ball = float(matches.cell(row=rowx,column=6).value)
        k=int(ball)
        if k<6:
            if wkt>=2:
                w=2
            else: w=wkt
            innings = matches.cell(row=rowx,column=5).value
            
            #innings change data
            if innings!=i:
                wkt,w = 0,0
                i = innings
            
            #over change data 
            if round(ball-k,1) ==0.1:
                bs=matches.cell(row=rowx,column=9).value
                bns=matches.cell(row=rowx,column=10).value
                bw=matches.cell(row=rowx,column=11).value
                        
                update_batsman(bs,"m"+str(k+1),1)
                update_batsman(bns,"m"+str(k+1),1)
                update_bowler(bw,"m("+str(k+1)+str(w)+")",1)
                blist={bs,bns}
            
            #player change data
            else:
                if bw!=matches.cell(row=rowx,column=11).value:
                    update_bowler(bw,"m("+str(k+1)+str(w)+")",1)
                
                bs=matches.cell(row=rowx,column=9).value
                bns=matches.cell(row=rowx,column=10).value
                if bs not in blist:
                    update_batsman(bs,"m"+str(k+1),1)
                if bns not in blist:
                    update_batsman(bns,"m"+str(k+1),1)
                blist={bs,bns}
                    
            #runs
            runs_off_bat=matches.cell(row=rowx,column=12).value
            extras=matches.cell(row=rowx,column=13).value
            update_batsman(bs,"t"+str(k+1),runs_off_bat)
            update_bowler(bw,"t("+str(k+1)+str(w)+")",runs_off_bat+extras)
            
            #wickets
            if matches.cell(row=rowx,column=19).value is not None:
                player_dismissed=matches.cell(row=rowx,column=20).value
                bw=matches.cell(row=rowx,column=11).value
                update_batsman(player_dismissed,"w"+str(k+1),1)
                update_bowler(bw,"w"+str(k+1),1)
                wkt+=1


In [22]:
wb_batsman.save("batsman_data.xlsx")
wb_bowler.save("bowler_data.xlsx")